In [2]:
## Import Library

import openai
from azure.core.credentials import AzureKeyCredential
from azure.identity import AzureDeveloperCliCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    HnswParameters,
    PrioritizedFields,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticSettings,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
)
from azure.storage.blob import BlobServiceClient

import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

import openai
import os
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError
import json

from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
#from langchain.retrievers import AzureCognitiveSearchRetriever
from langdetect import detect
from langchain.prompts import PromptTemplate
import re
# Create chain to answer questions
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Import Azure OpenAI
from langchain.llms import AzureOpenAI 
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

#import textwrap
import logging


In [3]:
"""#setting up credentials
os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "gptdemosearch" # replace with yours search service name
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "PcAZcXbX2hJsxMYExc2SnkMFO0D94p7Zw3Qzeu5WjYAzSeDMuR5O" # replace with your api key
os.environ["AZURE_INDEX_NAME"] = "sino-hr-chatbot" #"namfung-finance-chatbot" # 
# end setting up credentials"""

# setting up credentials
os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "acs-testing-sunny" # replace with yours search service name
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = "oygYftyrBXiWoDLoZatDKNSLFttn9frM6DE4XlSb7kAzSeBR01eY" # replace with your api key
os.environ["AZURE_INDEX_NAME"] = "your-index-name" #"namfung-finance-chatbot" # 
# end setting up credentials


In [4]:
def generate_prompt():
    prompt_template_string="""
    Follow exactly these 5 steps:
    1. Read the context below and aggregrate this data
    Context : {context}
    2. Answer the question using only this context
    3. Answer the question in less than 200 words
    4. Please provide the page number of the page where your answer is based on 
    5. Please provide the page number in the following output format: [Page: 1]
    
    User Question: {question}


    If you don't have any context and are unsure of the answer, reply that you don't know about this topic.
    """
    prompt_template = PromptTemplate(template = prompt_template_string, input_variables=["context", "question"])

    return prompt_template

def generate_prompt_chi():
    prompt_template_string="""
    指令：
    1. 你必须只根据以下文本的内容回答提问者的询问。
    2. 如果不懂得回答或文本没有资料，请回答“对不起，我不懂得回答这个问题。”
    3. 请以少於200字回答问题。
    4. 请在你的回答后提供你用以回答的文本的页数。格式示例：[Page: 1]
    
    文本：{context}
    #####

    问题：{question}

    """
    prompt_template = PromptTemplate(template = prompt_template_string, input_variables=["context", "question"])

    return prompt_template


In [5]:
def initialize_vector_store():
    # set up index name 
    index_name = os.environ["AZURE_INDEX_NAME"] 
    
    # set up openai environment
    os.environ["OPENAI_API_TYPE"] = "azure"
    os.environ["OPENAI_API_BASE"] = "https://pwcjay.openai.azure.com/"
    os.environ["OPENAI_API_VERSION"] = "2023-05-15"
    os.environ["OPENAI_API_KEY"] = "f282a661571f45a0bdfdcd295ac808e7"

    model: str = "text-embedding-ada-002"
    search_service = os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"]
    search_api_key = os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"]
    vector_store_address: str = f"https://{search_service}.search.windows.net"
    vector_store_password: str = search_api_key
    

    # define embedding model for calculating the embeddings
    model: str = "text-embedding-ada-002"
    embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, chunk_size=1)
    embedding_function = embeddings.embed_query

    # define schema of the json file stored on the index
    fields = [
            SimpleField(
                name="id",
                type=SearchFieldDataType.String,
                key=True,
                filterable=True,
            ),
            SearchableField(
                name="content",
                type=SearchFieldDataType.String,
                searchable=True,
            ),
            SearchField(
                name="content_vector",
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True,
                vector_search_dimensions=len(embedding_function("Text")),
                vector_search_configuration="default",
            ),
            SearchableField(
                name="metadata",
                type=SearchFieldDataType.String,
                searchable=True,
            ),
            # Additional field to store the title
            SearchableField(
                name="title",
                type=SearchFieldDataType.String,
                searchable=True,
            ),
            # Additional field for filtering on document source
            SimpleField(
                name="source",
                type=SearchFieldDataType.String,
                filterable=True,
            ),
            # Additional field for filtering on document source
            SimpleField(
                name="page",
                type=SearchFieldDataType.String,
                filterable=True,
            ),
            # Additional field for filtering on document source
            SimpleField(
                name="website_url",
                type=SearchFieldDataType.String,
                filterable=True,
            ),
        ]    
    
    vector_store: AzureSearch = AzureSearch(
        azure_search_endpoint=vector_store_address,
        azure_search_key=vector_store_password,
        index_name=index_name,
        embedding_function=embedding_function,
        fields=fields,
    )
    
    return vector_store

In [6]:
def azure_search_by_index(question, index_name):

    # set up openai environment
    os.environ["OPENAI_API_TYPE"] = "azure"
    os.environ["OPENAI_API_BASE"] = "https://pwcjay.openai.azure.com/"
    os.environ["OPENAI_API_VERSION"] = "2023-05-15"
    os.environ["OPENAI_API_KEY"] = "f282a661571f45a0bdfdcd295ac808e7"

    model: str = "text-embedding-ada-002"
    search_service = os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"]
    search_api_key = os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"]
    vector_store_address: str = f"https://{search_service}.search.windows.net"
    vector_store_password: str = search_api_key
    

    # define embedding model for calculating the embeddings
    model: str = "text-embedding-ada-002"
    embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=model, chunk_size=1)
    embedding_function = embeddings.embed_query

    # define schema of the json file stored on the index
    fields = [
            SimpleField(
                name="id",
                type=SearchFieldDataType.String,
                key=True,
                filterable=True,
            ),
            SearchableField(
                name="content",
                type=SearchFieldDataType.String,
                searchable=True,
            ),
            SearchField(
                name="content_vector",
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True,
                vector_search_dimensions=len(embedding_function("Text")),
                vector_search_configuration="default",
            ),
            SearchableField(
                name="metadata",
                type=SearchFieldDataType.String,
                searchable=True,
            ),
            # Additional field to store the title
            SearchableField(
                name="title",
                type=SearchFieldDataType.String,
                searchable=True,
            ),
            # Additional field for filtering on document source
            SimpleField(
                name="source",
                type=SearchFieldDataType.String,
                filterable=True,
            ),
            # Additional field for filtering on document source
            SimpleField(
                name="page",
                type=SearchFieldDataType.String,
                filterable=True,
            ),
            # Additional field for filtering on document source
            SimpleField(
                name="website_url",
                type=SearchFieldDataType.String,
                filterable=True,
            ),
        ]    
    
    vector_store: AzureSearch = AzureSearch(
        azure_search_endpoint=vector_store_address,
        azure_search_key=vector_store_password,
        index_name=index_name,
        embedding_function=embedding_function,
        fields=fields,
    )

    relevant_documentation = vector_store.similarity_search(query=question, k=1, search_type="similarity")
    
    context = "\n".join([doc.page_content for doc in relevant_documentation])[:10000]

    lang = detect(context)

    #print(doc)
    #print(context)
    #print(relevant_documentation)
    source = relevant_documentation[0].metadata['source']
    #page_no = relevant_documentation[0].metadata['page']
    website_url = relevant_documentation[0].metadata['website_url']
    
    page_no = ""
    for doc in relevant_documentation:
        page_no = page_no + "," + doc.metadata['page'] 
    
    #print(relevant_documentation[0])
    #print(source)
    #print(page_no)
    #print(website_url)
    #return str(context), source, website_url, lang, page_no
    # just return 10 documents (i.e. pages) if number of pages return from the search result > 10
    if len(relevant_documentation) > 10:
        relevant_documentation = relevant_documentation[0:9]
    else:
        relevant_documentation = relevant_documentation
    return relevant_documentation, source, website_url, lang, page_no


# helper function to extract page number
def extract_page_no(string):
    if "[Page" in string:
        print(re.findall('\[Page:.*\]', string)[0].split('Page:')[1])
        return re.findall('\[Page:.*\]', string)[0].split(':')[1].split("]")[0].strip()
    else:
        return "/"

def extract_answer(string):
    if "[Page" in string:
        return string.split("[Page")[0]
    else:
        return string


In [7]:
from langchain.chains import LLMChain
from langchain.llms import AzureOpenAI 
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import CosmosDBChatMessageHistory




In [8]:

# do not run, not use this for memory

def llm_pipeline(question):
    # set up index name 
    index_name = os.environ["AZURE_INDEX_NAME"] 

    # retrieve information from Azure Search
    relevant_docs, source, website_url, language, page_no = azure_search_by_index(question, index_name)

    #print(relevant_docs)

    # generate prompt without example

    if language == "en":
    #english prompt
        PROMPT = generate_prompt()
    else:
        PROMPT = generate_prompt_chi()

    os.environ["OPENAI_API_TYPE"] = "azure"
    os.environ["OPENAI_API_VERSION"] = "2023-05-15"
    os.environ["OPENAI_API_BASE"] = "https://pwcjay.openai.azure.com/"
    os.environ["OPENAI_API_KEY"] = "f282a661571f45a0bdfdcd295ac808e7"


    # use AzureChatOpenAI 
    llm = AzureChatOpenAI(deployment_name="gpt-35-16k", temperature=0,
                        openai_api_version="2023-05-15", openai_api_base="https://pwcjay.openai.azure.com/")


    # set up chat history database credentials
    os.environ["COSMOS_ENDPOINT"] = "https://gpt-demo-chat-history.documents.azure.com:443/"
    os.environ["COSMOS_KEY"] = "AZEhMpW4YD3t7iEMgp9at48S7f5ZjvnahUqJMYjMjMpH2QH2wiYBL97RdX7AqL3CMQcGGhbdAFHvACDbDDwMyA=="
    ENDPOINT = os.environ["COSMOS_ENDPOINT"]
    KEY = os.environ["COSMOS_KEY"]
    DATABASE_NAME = "sino_demo"
    CONTAINER_NAME = "sino_chat_history"
    sessionId = "101"
    user_id = "Sunny"

    history = CosmosDBChatMessageHistory(
        cosmos_endpoint = ENDPOINT,
        cosmos_database = DATABASE_NAME,
        cosmos_container = CONTAINER_NAME,
        credential = KEY,
        session_id = sessionId,
        user_id = user_id,
    )

    history.prepare_cosmos()



    # add chat memory
    memory = ConversationBufferMemory(
        llm = llm,
        output_key='answer',
        memory_key='chat_history',
        chat_memory=history,
        return_messages=True
    )

    chain = LLMChain(llm=llm, 
                    prompt=PROMPT,
                    memory=memory
                   #verbose=True
                    )

    output = chain.run({"context": relevant_docs, #"context": relevant_docs, 
        "question": question,
        })

In [9]:
def generate_prompt_with_history():
    prompt_template_string="""
    Follow exactly these 5 steps:
    1. Read the context below and aggregrate this data
    Context : {context}
    2. Answer the question using only this context and the chat history below
    3. Answer the question in less than 200 words
    4. Please provide the page number of the pages where your answer are based on at the end of your response
    5. Please provide the page numbers in the following output format: [Page: 1, 2, 3]
    
    Chat History: {chat_history}

    User Question: {question}


    If you don't have any context and are unsure of the answer, reply that you don't know about this topic.
    """
    prompt_template = PromptTemplate(template = prompt_template_string, input_variables=["context", "question", "chat_history"])

    return prompt_template

def generate_prompt_chi_with_history():
    prompt_template_string="""
    指令：
    1. 你必须只根据以下文本的内容及谈话记录回答提问者的询问。
    2. 如果不懂得回答或文本没有资料，请回答“对不起，我不懂得回答这个问题。”
    3. 请以少於200字回答问题。
    4. 请在你的回答后提供你用以回答的文本的页数。格式示例：[Page: 1]
    
    文本：{context}

    谈话记录：{chat_history}

    #####

    问题：{question}

    """
    prompt_template = PromptTemplate(template = prompt_template_string, input_variables=["context", "question", "chat_history"])

    return prompt_template


In [15]:
from langchain.chains import ConversationalRetrievalChain

def llm_pipeline_with_history(question):
    # set up index name 
    index_name = os.environ["AZURE_INDEX_NAME"] 

    os.environ["OPENAI_API_TYPE"] = "azure"
    os.environ["OPENAI_API_VERSION"] = "2023-05-15"
    os.environ["OPENAI_API_BASE"] = "https://pwcjay.openai.azure.com/"
    os.environ["OPENAI_API_KEY"] = "f282a661571f45a0bdfdcd295ac808e7"


    # retrieve information from Azure Search
    relevant_docs, source, website_url, language, page_no = azure_search_by_index(question, index_name)

    language = detect(question)

    if language == "en":
    #english prompt
        QA_CHAIN_PROMPT = generate_prompt_with_history()
    else:
        QA_CHAIN_PROMPT = generate_prompt_chi_with_history()

    # use AzureChatOpenAI 
    llm = AzureChatOpenAI(deployment_name="gpt-35-16k", temperature=0,
                        openai_api_version="2023-05-15", openai_api_base="https://pwcjay.openai.azure.com/")


    # set up chat history database credentials
    os.environ["COSMOS_ENDPOINT"] = "https://gpt-demo-chat-history.documents.azure.com:443/"
    os.environ["COSMOS_KEY"] = "AZEhMpW4YD3t7iEMgp9at48S7f5ZjvnahUqJMYjMjMpH2QH2wiYBL97RdX7AqL3CMQcGGhbdAFHvACDbDDwMyA=="
    ENDPOINT = os.environ["COSMOS_ENDPOINT"]
    KEY = os.environ["COSMOS_KEY"]
    DATABASE_NAME = "sino_demo"
    CONTAINER_NAME = "sino_chat_history"

    # session id, to be provided by frontend
    sessionId = "103"
    user_id = "Sunnytesting"

    history = CosmosDBChatMessageHistory(
        cosmos_endpoint = ENDPOINT,
        cosmos_database = DATABASE_NAME,
        cosmos_container = CONTAINER_NAME,
        credential = KEY,
        session_id = sessionId,
        user_id = user_id,
    )

    history.prepare_cosmos()

    vector_store =initialize_vector_store()

    retriever = vector_store.as_retriever()

    # add chat memory
    memory = ConversationBufferMemory(
        llm = llm,
        output_key='answer',
        memory_key='chat_history',
        chat_memory=history,
        return_messages=True
    )

    #QA_CHAIN_PROMPT = generate_prompt_with_history()

    # The chain
    chain = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
        verbose=True,
    )

    #QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

    output = chain({"question": question})

    #answer_value = output["answer"]

    #print(answer_value)
    
    return output


In [16]:
question = "METHODS TO DOCUMENT BA AND BE ?"

llm_pipeline_with_history(question)



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: What competency and capacity must a third-party certification body have to qualify for accreditation?
Assistant: To qualify for accreditation, a third-party certification body must demonstrate that it has the necessary competency and capacity. This includes having adequate resources, such as a sufficient number of employees and agents with relevant knowledge, skills, and experience to effectively examine compliance with food safety requirements, industry standards, and practices. The certification body must also have adequate financial resources for its operations. Additionally, the certification body must show that it has the capability to meet the applicable requirements of the accreditation process, including the ability to conduct audits, issue certificatio


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    Follow exactly these 5 steps:
    1. Read the context below and aggregrate this data
    Context :  
 Draft — Not for Implementation 
 
Contains Nonbinding Recommendations 
 
487  
488 1.  Preapproval Changes 
489  
490 For BA and BE studies, we recommend a singl e-dose, fasting study be performed.  Under 
491 certain circumstances, multiple-dose BA studies  (see section III.A.5) and/or food effect 
492 studies may be necessary (See the FDA guidance for industry Food-Effect Bioavailability 
493 and Fed Bioequivalence). Unconventional dosage forms (buccal, chewable, orally 
494 disintegrating, and sublingua l dosage forms) should be administered according to 
495 intended label use/instructi ons.  In addition, a BA study may be needed with the 
496 unconventional dosage form swallowed intact  to assess the impact of accidental 
497 swallowing of the intact pr


> Finished chain.

> Finished chain.


{'question': 'METHODS TO DOCUMENT BA AND BE ?',
 'chat_history': [HumanMessage(content='What competency and capacity must a third-party certification body have to qualify for accreditation?', additional_kwargs={}, example=False),
  AIMessage(content='To qualify for accreditation, a third-party certification body must demonstrate that it has the necessary competency and capacity. This includes having adequate resources, such as a sufficient number of employees and agents with relevant knowledge, skills, and experience to effectively examine compliance with food safety requirements, industry standards, and practices. The certification body must also have adequate financial resources for its operations. Additionally, the certification body must show that it has the capability to meet the applicable requirements of the accreditation process, including the ability to conduct audits, issue certifications, and address any deficiencies identified during the auditing process. The certification 

In [12]:
def retrieve_chat_history(history):
    i = 0
    for message in history.messages:
        if i % 2 == 0:
            print("Human: " + message.content)
        else:
            print("AI: " + message.content)
        i+=1


retrieve_chat_history(history)

NameError: name 'history' is not defined

In [ ]:
%pip show langchain

In [ ]:
%pip show azure-search-documents

In [ ]:
%pip install azure-cosmos

In [ ]:
output